<a href="https://colab.research.google.com/github/kterashi/niigata_lecture_2024/blob/master/Lec7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Machine Learning Applications to High-Energy Physics

## Event Classfication using Quantum Neural Networks

First, we consider the problem of classifying signal events from background events using kinematic features in the events, one of the most representative tasks for machine learning in HEP data analysis.

### Preparation of training data

In [ ]:
# First, get all the necessary libraries from the copy and import packages
import os
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

In [ ]:
!git clone -b master https://github.com/kterashi/niigata_lecture_2024

# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from collections import OrderedDict

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import TwoLocal, ZFeatureMap, ZZFeatureMap
from qiskit.primitives import Estimator, Sampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_optimization.applications import OptimizationApplication
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler as RuntimeSampler
from qiskit_ibm_runtime.accounts import AccountNotFoundError
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [3]:
# Read data from file
input_file = os.getcwd()+'/niigata_lecture_2024/data/SUSY_1K.csv'
df = pd.read_csv(input_file,
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

# Number of input features for training
feature_dim = 3

# Feature variables
if feature_dim == 3:
    selected_features = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    selected_features = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    selected_features = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

# Number of events used in the training and testing
train_size = 20
test_size = 20

df_sig = df.loc[df.isSignal==1, selected_features]
df_bkg = df.loc[df.isSignal==0, selected_features]

# Extract the samples
df_sig_train = df_sig.values[:train_size]
df_bkg_train = df_bkg.values[:train_size]
df_sig_test = df_sig.values[train_size:train_size + test_size]
df_bkg_test = df_bkg.values[train_size:train_size + test_size]
# The first train_size events contain SUSY signal and the last train_size events do not.
train_data = np.concatenate([df_sig_train, df_bkg_train])
# The first test_size events contain SUSY signal and the last test_size events do not.
test_data = np.concatenate([df_sig_test, df_bkg_test])

# Label
train_label = np.zeros(train_size * 2, dtype=int)
train_label[:train_size] = 1
test_label = np.zeros(train_size * 2, dtype=int)
test_label[:test_size] = 1

train_label_one_hot = np.zeros((train_size * 2, 2))
train_label_one_hot[:train_size, 0] = 1
train_label_one_hot[train_size:, 1] = 1
test_label_one_hot = np.zeros((test_size * 2, 2))
test_label_one_hot[:test_size, 0] = 1
test_label_one_hot[test_size:, 1] = 1

#datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
#datapoints_tr, class_to_label_tr = split_dataset_to_data_and_labels(training_input)

mms = MinMaxScaler((-1, 1))
norm_train_data = mms.fit_transform(train_data)
norm_test_data = mms.transform(test_data)

### State Preparation with Feature Map

In [ ]:
#feature_map = ZFeatureMap(feature_dimension=feature_dim, reps=1)
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1, entanglement='circular')
feature_map.decompose().draw('mpl')

### State Transformation with Variational Form

In [ ]:
ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry', 'rz'], entanglement_blocks='cz', entanglement='circular', reps=3)
#ansatz = TwoLocal(num_qubits=feature_dim, rotation_blocks=['ry'], entanglement_blocks='cz', entanglement='circular', reps=3)
ansatz.decompose().draw('mpl')

### Measurement and Model Output

In [ ]:
# Use Sampler instead of backend
sampler = Sampler()

maxiter = 300

optimizer = COBYLA(maxiter=maxiter, disp=True)

objective_func_vals = []
# Draw the value of objective function every time when the fit() method is called
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    #print('obj_func_eval =',obj_func_eval)

    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(objective_func_vals)
    plt.show()

vqc = VQC(num_qubits=feature_dim,
          feature_map=feature_map,
          ansatz=ansatz,
          loss="cross_entropy",
          optimizer=optimizer,
          callback=callback_graph,
          sampler=sampler)

### Execute with Simulator

In [ ]:
vqc.fit(norm_train_data, train_label_one_hot)

train_score = vqc.score(norm_train_data, train_label_one_hot)
test_score = vqc.score(norm_test_data, test_label_one_hot)

print(f'--- Classification Train score: {train_score} ---')
print(f'--- Classification Test score:  {test_score} ---')

## Event Classfication using Quantum Kernel Method

Now we try to tackle the same problme with quantum kernel method. First, implement a quantum circuit to encode input data as QuantumCircuit instance. You could use, e.g, ZFeatureMap or ZZFeatureMap as used above or something different.

You can change the number of qubits used, but it appears that FidelityQuantumKernel class that we use later will work better if the number of input features is the same as the number of qubits.

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1, entanglement='circular')
feature_map.decompose().draw('mpl')

Next step is to create a quantum circuit to calculate kernel matrix from the feature map defined above. Qiskit has an API (FidelityQuantumKernel class) to do that, and we use it here.

In [9]:
# FidelityQuantumKernel creates a Sampler instance internally
q_kernel = FidelityQuantumKernel(feature_map=feature_map)

We try to calculate the fidelity between the two inputs states, $|\langle0^{\otimes n}|U_{\rm{in}}^\dagger(x_1)U_{\rm{in}}(x_0)|0^{\otimes n}\rangle|^2$, with the FidelityQuantumKernel class. The circuit to do that is obtained by assigning input data to parameters in the feature_map.

In [ ]:
bind_params = dict(zip(feature_map.parameters, norm_train_data[0]))
feature_map_0 = feature_map.assign_parameters(bind_params)
bind_params = dict(zip(feature_map.parameters, norm_train_data[1]))
feature_map_1 = feature_map.assign_parameters(bind_params)

qc_circuit = q_kernel.fidelity.create_fidelity_circuit(feature_map_0, feature_map_1)
qc_circuit.decompose().decompose().draw('mpl')

Execute the circuit with simulator and calculate the probability of measuring 0 in all qubits after applying $U_{\rm{in}}^\dagger(x_1)U_{\rm{in}}(x_0)$ to the input $|0^{\otimes n}\rangle$ state.

In [ ]:
sampler = Sampler()

job = sampler.run(qc_circuit, shots=10000)

# quasi_dists[0] is the probability distribution of expected measured counts
fidelity = job.result().quasi_dists[0].get(0, 0.)
print(f'|<φ(x_1)|φ(x_0)>|^2 = {fidelity}')

The FidelityQuantumKernel allows us to visualize a kernel matrix obtained from the fidelity calculations. Here we make plots of the kernel matrices calculated from the training data alone, and from the training and test data.


In [ ]:
matrix_train = q_kernel.evaluate(x_vec=norm_train_data)
matrix_test = q_kernel.evaluate(x_vec=norm_test_data, y_vec=norm_train_data)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train), interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test), interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("validation kernel matrix")
plt.show()

Finally, the data are classified into signal and background using support vector machine implemented in sklearn package.

In [ ]:
qc_svc = SVC(kernel='precomputed') # Default value of hyperparameter (C) is 1
qc_svc.fit(matrix_train, train_label)

train_score = qc_svc.score(matrix_train, train_label)
test_score = qc_svc.score(matrix_test, test_label)

print(f'Precomputed kernel: Classification Train score: {train_score*100}%')
print(f'Precomputed kernel: Classification Test score:  {test_score*100}%')

## Quantum Machine Learning on Quantum Data

We have considered so far a classical ML task with input classical data. Next, we consider a bit more *quantum* ML task, in which quantum computer could be more advantageous than classical computer in the future.

What we aim here is a *Hamiltonian learning* task of estimating classical parameters of a given Hamiltonian from quantum states generated under the Hamiltonian. The transverse-field Ising model is considered, and the tasks is to estimate the strength of transverse field by learning VQE-generated ground states using quantum neural networks.

Let's begin VQE using the Ising-model Hamiltonian with varied transverse-field strengths.

In [ ]:
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

# VQE setup
num_qubits = 2

ansatz = TwoLocal(num_qubits, "ry", "cz", reps=3)  # Ry gates with trainable parameters and CZ for entanglement
optimizer = SLSQP(maxiter=1000)  # Classical optimizer
ansatz.decompose().draw('mpl')

The Ising model we consider has a parameter `alpha` that controls the mixture of transverse and longitudinal fields. `alpha=0` corresponds to a pure transverse field and `alpha=pi/2` a pure longitudinal field.

In [ ]:
# Use Estimator
estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)

# Ising model Hamiltonian with transverse and longitudinal fields
def get_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * np.sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * np.cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# Example:
J = 0.2
h = 1.2
alpha = np.pi/8
H = get_hamiltonian(L=num_qubits, J=J, h=h, alpha=alpha)

result = vqe.compute_minimum_eigenvalue(H)
#print(result)
print(f'VQE energy value = {result.optimal_value:.5f}')

Since the system is small, the exact ground state energy can be calcualted by diagonalizing the Hamiltonian.

In [ ]:
numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H)
ref_value = result.eigenvalue.real
print(f"Reference energy value = {ref_value:.5f}")

OK, now create a dataset of VQE ground states with different `h` values. For simplicity, the `alpha` parameter is set to 0 and only the transverse field is present.

In [ ]:
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.algorithms import NeuralNetworkRegressor

Nexp = 10   # number of experiments
Ntrain = 20  # number of training data per experiment
Ntest = 5
J = 0.2
alpha = 0

h_list = [np.random.rand()*0.25 for _ in range(Ntrain)]
#h_list = [np.random.rand() for _ in range(Ntrain)]
print(f'Input field strenghs = {h_list}')

In [ ]:
vqe_ansatz = TwoLocal(num_qubits, "ry", "cz", parameter_prefix='x')
optimizer = SLSQP(maxiter=1000)

estimator = Estimator()
vqe = VQE(estimator, vqe_ansatz, optimizer)

opt_vqe_energy = []
opt_vqe_params = []
for i in range(Ntrain):
    H = get_hamiltonian(L=num_qubits, J=J, h=h_list[i], alpha=alpha)
    result_vqe = vqe.compute_minimum_eigenvalue(H)
    opt_vqe_energy.append(result_vqe.optimal_value)
    opt_vqe_params.append(list(result_vqe.optimal_parameters.values()))
    print('VQE i =',i)

for i in range(Ntrain):
    print(f'VQE[{i}] energy value = {opt_vqe_energy[i]:.5f}')

Sanity check with exact diagonalizatiion

In [ ]:
numpy_solver = NumPyMinimumEigensolver()
for i in range(Ntrain):
    H = get_hamiltonian(L=num_qubits, J=J, h=h_list[i], alpha=alpha)
    result = numpy_solver.compute_minimum_eigenvalue(operator=H)
    ref_value = result.eigenvalue.real
    print(f"Reference[{i}] energy value = {ref_value:.5f}")

Now construct a quantum circuit with the optimized VQE circuit for generating ground states at the beginning and a QNN ansatz for learning the state at the end.

In [ ]:
from qiskit_machine_learning.circuit.library import QNNCircuit

# Observable : Pauli Z for the first qubit
pauli = 'I' * (num_qubits - 1)
pauli += 'Z'
obs = SparsePauliOp([pauli],coeffs=2.)

# Callback function to store loss values
def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)

# QNN setup with RealAmplitudes ansatz
nlayer = 3  # number of CX-RY layers

result_exp = []
regressor_exp = []
objective_func_vals_exp = []

# Repeat Nexp times
for iexp in range(Nexp):

    qnn_ansatz = RealAmplitudes(num_qubits=num_qubits, reps=nlayer, parameter_prefix='theta')

    '''
    qnn_ansatz = QuantumCircuit(num_qubits)
    theta = ParameterVector('theta')
    def new_theta():
        theta.resize(len(theta) + 1)
        return theta[-1]
    for il in range(0, nlayer):
        for i in range(0, num_qubits-1):
            qnn_ansatz.rzz(new_theta(),i,i+1)
        for i in range(0, num_qubits):
            qnn_ansatz.rx(new_theta(),i)
    '''

    qc = QuantumCircuit(num_qubits)
    qc.compose(vqe_ansatz, inplace=True)
    qc.compose(qnn_ansatz, inplace=True)

    # Random initial parameters within [0, pi]
    initial_weights = np.random.rand((nlayer+1)*num_qubits)*np.pi
    #initial_weights = np.random.rand((2*num_qubits-1)*nlayer)*np.pi

    # Use EstimatorQNN class
    qnn = EstimatorQNN(
        circuit = qc,
        input_params = vqe_ansatz.parameters,
        weight_params = qnn_ansatz.parameters,
        observables = obs
    )

    # Use NeuralNetworkRegressor for regression task with COBYLA optimizer
    regressor = NeuralNetworkRegressor(
        neural_network = qnn,
        loss = "squared_error",
        optimizer = SLSQP(maxiter=1000),
        warm_start = True,
        initial_point = initial_weights,
        callback = callback_graph
    )

    objective_func_vals = []
    result_regres = regressor.fit(np.array(opt_vqe_params),np.array(h_list))
    result_exp.append(result_regres)
    regressor_exp.append(regressor)
    objective_func_vals_exp.append(objective_func_vals)

    print(f'iexp = {iexp}')

Make the plot of loss function values.

In [ ]:
fig = plt.figure(facecolor="w")
ax = fig.add_subplot(1, 1, 1)
plt.title('Objective function value against iteration')
plt.xlabel("Iteration")
plt.ylabel("Objective function value")
for iexp in range(Nexp):
    plt.plot(range(len(objective_func_vals_exp[iexp])), objective_func_vals_exp[iexp])
#plt.legend()
plt.show()

See the correlation between the true and predicted transverse-field strengths from the optimized QNN model. The plot shows only the training data.

In [ ]:
train_pred_exp = []
for iexp in range(Nexp):
    train_pred = regressor_exp[iexp].predict(np.array(opt_vqe_params))
    train_pred_exp.append(train_pred)
    plt.scatter(h_list, train_pred, label='training')
plt.title('True vs Predicted values')
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.xlim(-0.05,0.3)
plt.ylim(-0.15,0.35)
plt.plot([-0.2,1.2],[-0.2,1.2],'k--')
plt.show()